In [1]:
import pandas as pd

A = pd.read_csv("data/A.csv").values
import numpy as np

x = np.array(A[:, 0:3].astype(np.float32))
x = np.hstack([x, np.ones([len(x[:, 0]), 1])])

y = np.array(A[:, 3:5].astype(np.float32))
from homography import Transformer

t = Transformer()
t.fit_homography(x, y)
import pickle

with open('./transformer.pickle', 'wb') as f:
    pickle.dump(t, f)

In [2]:
import pandas as pd

A = pd.read_csv("data/units_6100_gt.csv")

In [3]:
camera = A.loc[A['name']=='Camera']
c_x, c_y = float(camera['pos.x']), float(camera['pos.y'])

In [4]:
A.drop(A[A['name']=='Camera'].index, inplace=True)

In [5]:
A['pos.x'] -= c_x
A['pos.y'] -= c_y
A

,name,unit_type,owner,pos.x,pos.y,pos.z,radius,is_on_screen,feature_x,feature_y,wbs_expected
1,Barracks,21,2,-5.960938,2.410156,11.991103,1.8125,True,64.416667,102.291667,True
2,Bunker,24,1,3.039062,12.410156,9.989228,1.8125,True,160.416667,-4.375000,False
3,Bunker,24,2,-8.960938,4.410156,11.989014,1.8125,True,32.416667,80.958333,True
4,Bunker,24,2,-2.960938,-0.589844,11.985962,1.8125,True,96.416667,134.291667,True
5,Bunker,24,1,7.039062,10.410156,9.987030,1.8125,True,203.083333,16.958333,True
6,Factory,27,1,3.039062,21.410156,9.989228,1.8125,True,160.416667,-100.375000,False
7,Factory,27,2,3.039062,-4.589844,11.985962,1.8125,True,160.416667,176.958333,True
8,Factory,27,1,5.039062,16.410156,9.989228,1.8125,True,181.750000,-47.041667,False
9,FactoryTechLab,39,2,5.539062,-5.089844,11.985962,1.1250,True,187.083333,182.291667,True
10,LabMineralField,665,16,-19.460938,9.410156,7.988098,1.1250,True,-79.583333,27.625000,False


In [6]:
from homography import Transformer
import pickle

with open("./transformer.pickle", 'rb') as f:
    t = pickle.load(f)

uv = t.homography_transform(A[['pos.x', 'pos.y', 'pos.z']].values)

In [7]:
A['u'] = uv[:, 0]
A['v'] = uv[:, 1]

In [8]:
A['w'] = A['radius']*t.x_scale
A['h'] = A['radius']*t.y_scale

In [9]:
yolo_data = A[['unit_type', 'u', 'v', 'w', 'h']].values

In [10]:
import pybboxes as pbx
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("data/tick_6100.png")

for box in yolo_data.tolist():
    b = pbx.YoloBoundingBox(box[1], box[2], box[3], box[4], (t.image_size, t.image_size))
    b = pbx.convert_bbox(b, to_type="voc")
    cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 1)


cv2.imwrite("tick_6100_boxes.png", img)

True